In [2]:
import geopandas as gpd 
import geemap
import rasterio
import ee 

In [3]:
ee.Authenticate()
ee.Initialize()

# Manejo de archivos vectoriales

In [4]:
edom= gpd.read_file(r'C:\Users\egonz\Documents\Curso\Variables\Edomex\Edomex.shp')

In [16]:
# Simplificar las geometrías con una tolerancia deseada. Ajusta este valor según sea necesario.
# La tolerancia estará en las mismas unidades que el sistema de coordenadas del shapefile.
gdf_simplified = edom.simplify(tolerance=1.0)

# Guardar el shapefile simplificado
#gdf_simplified.to_file(r'C:\Users\egonz\Documents\Curso\Variables\Edomex\prueba.shp')

In [4]:
import os

# Crea un mapa interactivo
#Map = geemap.Map()

# Ruta al archivo shapefile
shp_file_path = r'C:\Users\egonz\Documents\Curso\Variables\Edomex\Edomex.shp'

# Comprueba si el archivo shapefile existe
if not os.path.exists(shp_file_path):
    raise FileNotFoundError("El archivo shapefile no se encontró.")

# Convierte el shapefile a un Earth Engine FeatureCollection
ee_object = geemap.shp_to_ee(shp_file_path)

# Añade la capa al mapa
#Map.addLayer(ee_object, {}, 'Nombre de la capa')

# Muestra el mapa
#Map

# Manejo de archivos raster

# Temperatura

In [6]:
# Recortar la imagen para evitar trabajar con bits innecesarios

mask= r"C:\Users\egonz\Documents\Curso\Variables\Edomex\Edomex.shp"
temp= r"C:\Users\egonz\Documents\Curso\Variables\Insumos\Temp.tif"
out= r"C:\Users\egonz\Documents\Curso\Variables\TempClip.tif"

#tempClip= geemap.clip_image(temp,mask,out)

In [5]:
import numpy as np

## Funcional no función

In [34]:
""" import rasterio
#import numpy as np

# Abre el archivo raster original
with rasterio.open(r'C:\Users\egonz\Documents\Curso\Variables\TempClip.tif') as src:
    raster_data = src.read(1)  # Lee la primera banda
    profile = src.profile

# Define tus reglas de reclasificación, por ejemplo:
# Si el valor de píxel está entre 0 y 100, se reclasifica a 1
# Si el valor de píxel está entre 101 y 200, se reclasifica a 2
# Si el valor de píxel es mayor que 200, se reclasifica a 3
reclassified_data = np.where((raster_data > 0) & (raster_data <= 100), 1, raster_data)
reclassified_data = np.where((raster_data > 100) & (raster_data <= 200), 2, reclassified_data)
reclassified_data = np.where(raster_data > 200, 3, reclassified_data)

# Guarda el raster reclasificado
with rasterio.open(r'C:\Users\egonz\Documents\Curso\Variables\TempReclass.tif', 'w', **profile) as dst:
    dst.write(reclassified_data, 1)
 """

## Funcion recursiva

In [9]:
def reclasifica_raster(nombre_archivo, intervalos_valores, ruta_salida):
    with rio.open(nombre_archivo) as src:
        raster = src.read(1)
        raster_reclasificado = np.copy(raster)
        
        # Iterar sobre cada intervalo y su nuevo valor
        for intervalo in intervalos_valores:
            # Verificar que la tupla contenga exactamente dos elementos
            if len(intervalo) != 2:
                raise ValueError("Cada tupla debe tener exactamente dos elementos (intervalo, nuevo_valor)")
            rango, nuevo_valor = intervalo

            # Aplicar el nuevo valor a los píxeles dentro del intervalo
            raster_reclasificado = np.where(
                (raster > rango[0]) & (raster <= rango[1]),
                nuevo_valor,
                raster_reclasificado
            )

        # Escribir el raster reclasificado en la ruta de salida
        srcprof = src.profile.copy()
        srcprof.update(dtype=raster_reclasificado.dtype)
        with rio.open(ruta_salida, 'w', **srcprof) as dst:
            dst.write(raster_reclasificado, 1)

In [41]:
nombre_archivo= r'C:\Users\egonz\Documents\Curso\Variables\TempClip.tif'
lista_intervalos_valores= [
    ((-np.inf,10),np.nan),
    ((10,10),10),
    ((10,12),20),
    ((12,15),70),
    ((15, 22), 100),
    ((22,28),85),
    ((28,32),70),
    ((32,40),20),
    ((40,40),10),
    ((40,np.inf),np.nan)
]
ruta_salida= r'C:\Users\egonz\Documents\Curso\Resultados\VariablesRes\TempReclas.tif'

In [44]:
#reclasifica_raster(nombre_archivo=nombre_archivo, intervalos_valores=lista_intervalos_valores, ruta_salida=ruta_salida)

# Precipitación

In [45]:
precip= r'C:\Users\egonz\Documents\Curso\Variables\Insumos\PrecipAnual.tif'
reclass= [
    ((-np.inf,400),10),
    ((400,500),20),
    ((500,600),75),
    ((600,800),100),
    ((800,1200),95),
    ((1200,1400),75),
    ((1400,1600),20),
    ((1600,np.inf),10)
]
out= r'C:\Users\egonz\Documents\Curso\Resultados\VariablesRes\PrecipReclass.tif'

In [46]:
#reclasifica_raster(nombre_archivo=precip, intervalos_valores=reclass, ruta_salida=out)

In [48]:
pr= r'C:\Users\egonz\Documents\Curso\Resultados\VariablesRes\PrecipReclass.tif'
out= r'C:\Users\egonz\Documents\Curso\Variables\PrecipRec_Clip.tif'

In [49]:
geemap.clip_image(pr,mask,out)

# Altura

In [5]:
Map= geemap.Map()

In [10]:
rut= r'C:\Users\egonz\Documents\Curso\Variables\Edomex\Edomex_gcs.shp'
edom= ee.FeatureCollection('projects/ee-emiliohub4/assets/Edomex')

In [32]:
dataset = ee.Image('USGS/SRTMGL1_003')
elev= dataset.select('elevation')
slope = ee.Terrain.slope(elev)

edmxElev= elev.clip(edom)

vis={'min': 1200, 'max': 2900}

Map.addLayer(edmxElev,vis,'elev')
Map

Map(bottom=29215.0, center=[19.150357455407484, -98.94288977912147], controls=(WidgetControl(options=['positio…

In [11]:
edom= edom.geometry()

In [44]:
""" image = elev.unmask()
filename= r'C:\Users\egonz\Documents\Curso\Resultados\VariablesRes\Altura.tif'
geemap.ee_export_image(
    image, filename=filename, scale=100, region=edom, file_per_band=False
) """

Generating URL ...
Please wait ...
Data downloaded to C:\Users\egonz\Documents\Curso\Resultados\VariablesRes\Altura.tif


In [53]:
import numpy as np
import rasterio as rio

In [55]:
ruta= r'C:\Users\egonz\Documents\Curso\Resultados\VariablesRes\Altura.tif'
file= r'C:\Users\egonz\Documents\Curso\Resultados\VariablesRes\AlturaReclass.tif'
ls=[
     ((1200,2200),100),
     ((2200,2700),95),
     ((0,1200),75),
     ((2700,2900),20),
     ((1200,2200),100),
     ((2900,np.inf),10) 
]


In [56]:
""" reclasifica_raster(ruta,ls,file) """

In [57]:
ras= r'C:\Users\egonz\Documents\Curso\Resultados\VariablesRes\AlturaReclass.tif' 
mask= r'C:\Users\egonz\Documents\Curso\Variables\Edomex\Edomex_gcs.shp'
out= r'C:\Users\egonz\Documents\Curso\Resultados\VariablesRes\AltRclsClip.tif'

geemap.clip_image(ras,mask,out)

# Pendiente

In [4]:
from numpy import tan

In [7]:
import ee
ee.Authenticate()
ee.Initialize()

In [12]:
dem = ee.Image('NASA/NASADEM_HGT/001').select('elevation')
slope = ee.Terrain.slope(dem)

#Debemos tener los valores de pendiente en porcentaje, por lo que se procede a convertir de grados a porcentaje
percent= slope.divide(180).multiply(np.pi).tan().multiply(100).rename('percent')


edmxSlp= percent.clip(edom)
""" m = geemap.Map()
m.add_layer(edmxSlp, {'min': 0, 'max': 90}, 'Slope')
m.centerObject(edmxSlp)
 """

" m = geemap.Map()\nm.add_layer(edmxSlp, {'min': 0, 'max': 90}, 'Slope')\nm.centerObject(edmxSlp)\n "

In [18]:
image = percent.unmask()
filename= r'C:\Users\egonz\Documents\Curso\Resultados\VariablesRes\SlopePerc.tif'
geemap.ee_export_image(
    image, filename=filename, scale=100, region=edom, file_per_band=False
)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\egonz\Documents\Curso\Resultados\VariablesRes\SlopePerc.tif


In [22]:
import rasterio as rio

In [28]:
rut= r'C:\Users\egonz\Documents\Curso\Resultados\VariablesRes\SlopePerc.tif'
lis= [
    ((0,8),100),
    ((8,15),95),
    ((15,30),80),
    ((30,40),20),
    ((40,45),20),
    ((45,np.inf),10)
]
out= r'C:\Users\egonz\Documents\Curso\Resultados\VariablesRes\SlopeRec.tif'
reclasifica_raster(rut,lis,out)

In [31]:
mask= r'C:\Users\egonz\Documents\Curso\Variables\Edomex\Edomex_gcs.shp'

In [32]:
ras= r'C:\Users\egonz\Documents\Curso\Resultados\VariablesRes\SlopeRec.tif'
out= r'C:\Users\egonz\Documents\Curso\Resultados\VariablesRes\ClipEdomexSlope.tif'
geemap.clip_image(ras,mask,out)

# Preprocesamiento de datos vectoriales

In [5]:
rut= r'C:\Users\egonz\Documents\Curso\Variables\Insumos\MAPAS_BASE\edomex_eda_utm84_corregido_final.shp'
edaf= gpd.read_file(rut)
edaf.head(2)

,AREA,PERIMETER,EDAEDOM_,EDAEDOM_ID,EDOMEXF_,EDOMEXF_ID,MEXEDA6_,MEXEDA6_ID,SUEL1,SUEL2,TEXT,FASFIS,FASQUIM,SUEL1_88,SUEL2_88,S1WRB_2006,S1WRB_2015,geometry
0,0.000354,0.248406,2,1,2,0,250,249,Vp,I,3,LITICA,NO,VRe,LPq,VReu,VRha,"POLYGON ((407506.691 2241953.964, 407595.012 2..."
1,0.000248,0.073757,3,2,2,0,246,245,Wm,NaN,2,DURICA PROFUNDA,NO,VRe,NaN,VRmo,VRha,"POLYGON ((405349.892 2242999.570, 405418.145 2..."


In [36]:
filtroSuelos= pd.DataFrame({
    'Nom': ['ANh',
            'ANm',
            'ANu',
            'PHc',
            'PHg',
            'PHh',
            'PHl',
            'VRe',
            'FLc',
            'FLd',
            'FLe',
            'CMcc',
            'CMcr',
            'CMdy',
            'CMeu',
            'CMfl',
            'CMhu',
            'CMvr',
            'HSe',
            'RGc',  
            'RGd', 
            'RGe',
            'ACh', 
            'ACu',
            'LVf',
            'LVh',
            'LVk',
            'LVv',
            'LVx',
            'LPk',
            'LPq',
            'LPu',
            'SCg',  
            'SCh',  
            'SCm', 
            'GLe',   
            'GLm',  
            'GLu'],

    'Suelo': ['Andosol Ócrico',
              'Andosol Mólico',
              'Andosol Húmico',
              'Feozem Calcárico',
              'Feozem Gléyico',
              'Feozem Háplico',
              'Feozem Lúvico',
              'Vertisol Eútrico',
              'Fluvisol Calcárico',
              'Fluvisol Dístrico',
              'Fluvisol Eútrico',
              'Cambisol Cálcico',
              'Cambisol Crómico',
              'Cambisol Dístrico',
              'Cambisol Eútrico',
              'Cambisol Ferrálico',
              'Cambisol Húmico',
              'Cambisol Vértico',
              'Histosol Eútrico',
              'Regosol Calcárico', 
              'Regosol Dístrico', 
              'Regosol Eútrico',
              'Acrisol Órtico', 
              'Acrisol Húmico',
              'Luvisol Férrico',
              'Luvisol Háplico',
              'Luvisol Cálcico',
              'Luvisol Vértico',
              'Luvisol Crómico',
              'Leptosol Réndzico',
              'Leptosol Lítico',
              'Leptosol Úmbrico',
              'Solonchak Gléyico',
              'Solonchak Hálico',
              'Solonchak Mólico',
              'Gleysol Eútrico',
              'Gleysol Mólico',
              'Gleysol Úmbrico'],

     'Valor': [
         100,
         100,
         100,
         100,
         100,
         100,
         100,
         95,
         90,
         90,
         90,
         80,
         80,
         80,
         80,
         80,
         80,
         80,
         75,
         70,
         70,
         70,
         50, 
         50,
         45,
         45,
         45,
         45,
         45,
         20,
         20,
         20,
         10,
         10,
         10,
         10,
         10,
         10]   
})

# No hay planosol

In [27]:
#No hay Planosol. Se busco en el dataframe original y no se encuentran datos 
#edaf[edaf['SUEL2_88']].str.contains('PLm' o 'PLe', case=False)
#edaf[edaf['SUEL2_88']].str.contains('PLm' o 'PLe', case=False)

In [37]:
filtroSuelos

,Nom,Suelo,Valor
0,ANh,Andosol Ócrico,100
1,ANm,Andosol Mólico,100
2,ANu,Andosol Húmico,100
3,PHc,Feozem Calcárico,100
4,PHg,Feozem Gléyico,100
5,PHh,Feozem Háplico,100
6,PHl,Feozem Lúvico,100
7,VRe,Vertisol Eútrico,95
8,FLc,Fluvisol Calcárico,90
9,FLd,Fluvisol Dístrico,90


In [39]:
# Realizar el merge
resultado = pd.merge(filtroSuelos, edaf, left_on='Nom', right_on='SUEL1_88', how='inner')

In [40]:
resultado

,Nom,Suelo,Valor,AREA,PERIMETER,EDAEDOM_,EDAEDOM_ID,EDOMEXF_,EDOMEXF_ID,MEXEDA6_,...,SUEL1,SUEL2,TEXT,FASFIS,FASQUIM,SUEL1_88,SUEL2_88,S1WRB_2006,S1WRB_2015,geometry
0,ANh,Andosol Ócrico,100,0.000024,0.034118,1032,1031,2,0,2480,...,To,NaN,2,NO,NO,ANh,NaN,ANha,ANha,"POLYGON ((381956.212 2210443.397, 381977.788 2..."
1,ANh,Andosol Ócrico,100,0.000192,0.130367,1059,1058,2,0,2480,...,To,NaN,2,NO,NO,ANh,NaN,ANha,ANha,"POLYGON ((382012.394 2207875.437, 382016.542 2..."
2,ANh,Andosol Ócrico,100,0.000083,0.044501,1224,1223,2,0,2874,...,To,I,2,PEDREGOSA,NO,ANh,LPq,ANha,ANha,"POLYGON ((389740.285 2206475.072, 389755.869 2..."
3,ANh,Andosol Ócrico,100,0.000005,0.009608,1241,1240,2,0,2924,...,To,I,2,PEDREGOSA,NO,ANh,LPq,ANha,ANha,"POLYGON ((391319.327 2206292.352, 391368.165 2..."
4,ANh,Andosol Ócrico,100,0.001903,0.916722,1252,1251,2,0,2964,...,To,NaN,2,NO,NO,ANh,NaN,ANha,ANha,"POLYGON ((391428.927 2205421.137, 391443.250 2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7656,GLm,Gleysol Mólico,10,0.000001,0.007169,6012,6011,2,0,8807,...,Gm,Oe,2,NO,LIGERAMENTE SALINO,GLm,HSs,GLmo,GLmo,"POLYGON ((505419.901 2124835.896, 505777.611 2..."
7657,GLu,Gleysol Úmbrico,10,0.000034,0.047648,2308,2307,2,0,4618,...,Gh,Zm,3,DURICA PROFUNDA,NO,GLu,SCm,GLhu,GLhu,"POLYGON ((483336.878 2188845.410, 483374.441 2..."
7658,GLu,Gleysol Úmbrico,10,0.000024,0.058401,2482,2481,2,0,4839,...,Gh,Zm,3,DURICA PROFUNDA,SODICA,GLu,SCm,GLhu,GLhu,"POLYGON ((487220.531 2186516.813, 487264.454 2..."
7659,GLu,Gleysol Úmbrico,10,0.000020,0.025365,2495,2494,2,0,4857,...,Gh,Zm,3,DURICA PROFUNDA,SODICA,GLu,SCm,GLhu,GLhu,"POLYGON ((485572.448 2185433.265, 485540.517 2..."


In [41]:
resultado.keys()

Index(['Nom', 'Suelo', 'Valor', 'AREA', 'PERIMETER', 'EDAEDOM_', 'EDAEDOM_ID',
       'EDOMEXF_', 'EDOMEXF_ID', 'MEXEDA6_', 'MEXEDA6_ID', 'SUEL1', 'SUEL2',
       'TEXT', 'FASFIS', 'FASQUIM', 'SUEL1_88', 'SUEL2_88', 'S1WRB_2006',
       'S1WRB_2015', 'geometry'],
      dtype='object')

In [43]:
resultado.SUEL1.unique()

array(['To', 'Tm', 'Th', 'Hc', 'Hg', 'Hh', 'Hl', 'Vp', 'Wm', 'We', 'Vc',
       'Wh', 'Jc', 'Jd', 'Je', 'Oe', 'Rc', 'Rv', 'Rd', 'Rg', 'Re', 'Ao',
       'Ah', 'Lf', 'Lo', 'Lv', 'Lk', 'Lc', 'E', 'Lg', 'I', 'U', 'Zg',
       'Zo', 'Zm', 'Gv', 'Ge', 'Gm', 'Gh'], dtype=object)

In [50]:
res= resultado[['Nom','Suelo','Valor','AREA','PERIMETER','SUEL1','SUEL2','TEXT','FASFIS','FASQUIM','SUEL1_88','SUEL2_88','S1WRB_2006','S1WRB_2015','geometry']]

In [51]:
res

,Nom,Suelo,Valor,AREA,PERIMETER,SUEL1,SUEL2,TEXT,FASFIS,FASQUIM,SUEL1_88,SUEL2_88,S1WRB_2006,S1WRB_2015,geometry
0,ANh,Andosol Ócrico,100,0.000024,0.034118,To,NaN,2,NO,NO,ANh,NaN,ANha,ANha,"POLYGON ((381956.212 2210443.397, 381977.788 2..."
1,ANh,Andosol Ócrico,100,0.000192,0.130367,To,NaN,2,NO,NO,ANh,NaN,ANha,ANha,"POLYGON ((382012.394 2207875.437, 382016.542 2..."
2,ANh,Andosol Ócrico,100,0.000083,0.044501,To,I,2,PEDREGOSA,NO,ANh,LPq,ANha,ANha,"POLYGON ((389740.285 2206475.072, 389755.869 2..."
3,ANh,Andosol Ócrico,100,0.000005,0.009608,To,I,2,PEDREGOSA,NO,ANh,LPq,ANha,ANha,"POLYGON ((391319.327 2206292.352, 391368.165 2..."
4,ANh,Andosol Ócrico,100,0.001903,0.916722,To,NaN,2,NO,NO,ANh,NaN,ANha,ANha,"POLYGON ((391428.927 2205421.137, 391443.250 2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7656,GLm,Gleysol Mólico,10,0.000001,0.007169,Gm,Oe,2,NO,LIGERAMENTE SALINO,GLm,HSs,GLmo,GLmo,"POLYGON ((505419.901 2124835.896, 505777.611 2..."
7657,GLu,Gleysol Úmbrico,10,0.000034,0.047648,Gh,Zm,3,DURICA PROFUNDA,NO,GLu,SCm,GLhu,GLhu,"POLYGON ((483336.878 2188845.410, 483374.441 2..."
7658,GLu,Gleysol Úmbrico,10,0.000024,0.058401,Gh,Zm,3,DURICA PROFUNDA,SODICA,GLu,SCm,GLhu,GLhu,"POLYGON ((487220.531 2186516.813, 487264.454 2..."
7659,GLu,Gleysol Úmbrico,10,0.000020,0.025365,Gh,Zm,3,DURICA PROFUNDA,SODICA,GLu,SCm,GLhu,GLhu,"POLYGON ((485572.448 2185433.265, 485540.517 2..."
